In [ ]:
import pandas as pd
import numpy as np
import re, html
from bs4 import BeautifulSoup as BS, NavigableString, SoupStrainer
from html_table_parser import parser_functions

In [ ]:
import collections
if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

parser_d0350 = SoupStrainer("section-2")
section2_pattern = re.compile(r"<SECTION-2((?!<SECTION-2)[\S\s\n])*?(D-0-3-5-0)[\S\s\n]*?</SECTION-2>")

In [ ]:
# 파일시스템에서 xml 문서들을 불러오기 

import os

path = 'C:/Users/Samsung/Documents/KPMG/kic/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xml')] 

path_line = []
for i in file_list_py:

    path= 'C:/Users/Samsung/Documents/KPMG/kic/'+ i
    path_line.append(path)

#print(path_line) 

### 텍스트 확인

In [ ]:
df_base = pd.DataFrame(columns=['full_text'])

for i in range(len(path_line)):
    # 인코딩 방식이 다른 문헌의 경우 예외를 적용하도록 함.
    try : 
        with open(path_line[i], 'r', encoding="cp949") as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")
    except:
        with open(path_line[i], 'r', encoding="utf-8",errors='ignore') as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")

    # xml 문서에서 텍스트 형태로 변환하기  
    txt_line = [list(text.stripped_strings) for text in remark_page.find_all(recursive=False) if list(text.stripped_strings) != [] and 'border="1"' not in text.prettify()]
    raw_txt = sum(txt_line, [])
    txt = "\n".join(raw_txt)
    df_base = df_base.append(pd.DataFrame([txt], columns=['full_text']))


In [ ]:
base_report = df_base.reset_index(inplace=False).drop(columns=['index'])
base_report.to_csv("base_report(\n).csv")

### 테이블 확인 

In [ ]:
# 표 확인
table1 = remark_page.find('table', border=1)
pd.DataFrame(parser_functions.make2d(table1))

In [ ]:
with open( 'C:/Users/Samsung/Documents/KPMG/kic/20210510000378.xml', 'r', encoding="cp949", errors='ignore') as data_xml:
    dsd_xml = data_xml.read()
    dsd_xml = dsd_xml.replace('&cr;', '&#13;')
    dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
    dsd_xml = html.unescape(dsd_xml)
    section2_section = section2_pattern.search(dsd_xml)
    section2_section = section2_section.group()
    
remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")

[pd.DataFrame(parser_functions.make2d(table)) for table in remark_page.find_all('table', border=1, recursive=False)]

In [ ]:
df_table = pd.DataFrame(columns=['full_text'])

for i in range(len(path_line)):
    # 인코딩 방식이 다른 문헌의 경우 예외를 적용하도록 함.
    try : 
        with open(path_line[i], 'r', encoding="cp949") as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")
    except:
        with open(path_line[i], 'r', encoding="utf-8",errors='ignore') as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")

    # xml 문서에서 표로 변환하기  
    table_line = [pd.DataFrame(parser_functions.make2d(table)) for table in remark_page.find_all('table', border=1,recursive=False)]
    raw_txt = sum( txt_line, [])
    txt = "   ".join(raw_txt)
    df_base = df_base.append(pd.DataFrame([txt], columns=['full_text']))


In [ ]:

#텍스트와 표를 동시 추출하는 방법 

df_textwithtable = pd.DataFrame(columns=['full_text'])

for i in range(1):

    # 인코딩 방식이 다른 문헌의 경우 예외를 적용하도록 함.
    try : 
        with open(path_line[i], 'r', encoding="cp949") as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")
    except:
        with open(path_line[i], 'r', encoding="utf-8",errors='ignore') as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")
    
    # 텍스트와 표를 동시 추출하기 
    # [list(text.stripped_strings) for text in remark_page.find_all(recursive=False) if list(text.stripped_strings) != [] and 'border="1"' not in text.prettify()]
    # [pd.DataFrame(parser_functions.make2d(table)) for table in remark_page.find_all('table', border=1,recursive=False)]
    
    doc = []
    for text, table in zip( remark_page.find_all(recursive=False),remark_page.find_all('table', border=1,recursive=False)) :
        
        if list(text.stripped_strings) != [] and 'border="1"'not in text.prettify():
            txt_line = list(text.stripped_strings)
            doc.extend(txt_line)
        
        table_line = [pd.DataFrame(parser_functions.make2d(table))]
        doc.extend(table_line)
        
    #raw_txt = sum(doc, [])
    
    os.chdir("c:/Users/Samsung/Documents/KPMG/info_file")
    with open(str(i) + '재무제표 파일.csv', 'w', newline='\n') as f:
        writer = csv.writer(f)
        writer.writerow(doc)   